In [1]:
import os

In [2]:
%pwd

'/Users/himanshu/Lung-Cancer-Prediction/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/himanshu/Lung-Cancer-Prediction'

In [5]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/Himanshusinghdev1/Lung-Cancer-Prediction.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "Himanshusinghdev1"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "e663963ebb08452411148b424509dcd751190e6a"

In [6]:
import tensorflow as tf

In [7]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2025-03-18 11:58:51.288639: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-03-18 11:58:51.289160: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [9]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [10]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/LungCancerDataset/lung_image_sets",
            mlflow_uri="https://dagshub.com/Himanshusinghdev1/Lung-Cancer-Prediction.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [11]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [ ]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )
    
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")


In [13]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2025-03-18 11:58:57,407: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-03-18 11:58:57,409: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-18 11:58:57,410: INFO: common: created directory at: artifacts]
Found 4500 images belonging to 3 classes.


2025-03-18 11:58:57.839807: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2025-03-18 11:58:57.952819: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


282/282 [==============================] - 81s 287ms/step - loss: 0.1112 - accuracy: 0.9609
[2025-03-18 12:00:19,173: INFO: common: json file saved at: scores.json]


2025/03/18 12:00:22 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2025-03-18 12:00:22,925: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 13). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: /var/folders/xs/_l9szjnn6v7gj7q43hlpc8h40000gn/T/tmph8_76yy0/model/data/model/assets
[2025-03-18 12:00:23,640: INFO: builder_impl: Assets written to: /var/folders/xs/_l9szjnn6v7gj7q43hlpc8h40000gn/T/tmph8_76yy0/model/data/model/assets]


2025/03/18 12:00:29 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/03/18 12:09:00 INFO mlflow.tracking._tracking_service.client: 🏃 View run rumbling-moose-62 at: https://dagshub.com/Himanshusinghdev1/Lung-Cancer-Prediction.mlflow/#/experiments/0/runs/2709a7cd32614d46b8cd8abd38c57054.
2025/03/18 12:09:00 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/Himanshusinghdev1/Lung-Cancer-Prediction.mlflow/#/experiments/0.


KeyboardInterrupt: 